In [ ]:
import math
import numpy as np

# Reading/Writing Data
import pandas as pd
import os
import csv

# For Progress Bar
from tqdm import tqdm

# Pytorch
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
import scipy

# %%
def same_seed(seed): 
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def train_valid_split(data_set, valid_ratio, seed):
    '''Split provided training data into training set and validation set'''
    valid_set_size = int(valid_ratio * len(data_set)) 
    train_set_size = len(data_set) - valid_set_size
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    return np.array(train_set), np.array(valid_set)

def predict(test_loader, model, device):
    model.eval() # Set your model to evaluation mode.
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)                        
        with torch.no_grad():                   
            pred = model(x)                     
            preds.append(pred.detach().cpu())   
    preds = torch.cat(preds, dim=0).numpy()  
    return preds

# %% [markdown]
# # Dataset

# %%
class myDataset(Dataset):
    '''
    x: Features.
    y: Targets, if none, do prediction.
    '''
    def __init__(self, x, y=None):
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)

    def __getitem__(self, idx):
        sample = self.x[idx]
        if len(sample.shape) == 2:  # If sample is 1D, reshape it to 2D
            sample = sample.unsqueeze(0)  # Add channel dimension for CNN input
        if self.y is None:
            return sample
        else:
            return sample, self.y[idx]

    def __len__(self):
        return len(self.x)

# %% [markdown]
# # Neural Network Model
# Try out different model architectures by modifying the class below.

# %%
class My_Model(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim):
        super(My_Model, self).__init__()
        # TODO: modify model's structure, be aware of dimensions. 
        neuron_num = [input_dim] + hidden_dim + [output_dim]
        
        layers1 = []
        for i in range(1, len(neuron_num)):
            layers1.append(nn.Linear(neuron_num[i-1], neuron_num[i]))
            if i < len(neuron_num) - 1:
                layers1.append(nn.BatchNorm1d(neuron_num[i]))
                layers1.append(nn.ReLU())
        self.fc1 = nn.Sequential(*layers1)

        
        layers2 = []
        for i in range(1, len(neuron_num)):
            layers2.append(nn.Linear(neuron_num[i-1], neuron_num[i]))
            if i < len(neuron_num) - 1:
                layers2.append(nn.BatchNorm1d(neuron_num[i]))
                layers2.append(nn.ReLU())
        self.fc2 = nn.Sequential(*layers2)
        
    def forward(self, x):
        output1 = self.fc1(x)
        output2 = self.fc2(x)
        output = torch.cat((output1, output2), dim=1)  # Concatenate along the feature dimension
        output = output.squeeze(1)

        return output

class mlp(nn.Module):
    # add freq middle of the model
    def __init__(self, input_dim, output_dim, hidden_dim):
        super(mlp, self).__init__()
        # TODO: modify model's structure, be aware of dimensions. 
        self.input_dim = input_dim

        self.fc1 = nn.Sequential(
            nn.Linear(input_dim, hidden_dim[0]),
            nn.BatchNorm1d(hidden_dim[0]),
            nn.ReLU(),
            
            nn.Linear(hidden_dim[0], hidden_dim[1]),
            nn.BatchNorm1d(hidden_dim[1]),
            nn.ReLU()
        )

        
        neuron_num = [input_dim] + hidden_dim + [output_dim]
        layers = []
        for i in range(3, len(neuron_num)):
            layers.append(nn.Linear(neuron_num[i-1], neuron_num[i]))
            if i < len(neuron_num) - 1:
                layers.append(nn.BatchNorm1d(neuron_num[i]))
                layers.append(nn.ReLU())
        self.fc2 = nn.Sequential(*layers)

    def forward(self, x):
        input1 = x[:, :self.input_dim]  # First 5 features
        freq = x[:, self.input_dim:]  # Last 5 features
        x = self.fc1(input1)
        middle = torch.cat((x, freq), dim=1)  # Concatenate along the feature dimension
        x = self.fc2(middle)    
        x = x.squeeze(1) # (B, 1) -> (B)
        return x
    
# %% [markdown]
# # Feature Selection
# Choose features you deem useful by modifying the function below.

# %%
def select_feat(train_data, valid_data, test_data, input_dim, output_dim, select_all=True):
    '''Selects useful features to perform regression'''
    y_train, y_valid, y_test = train_data[:,input_dim:], valid_data[:,input_dim:], test_data[:,input_dim:]
    raw_x_train, raw_x_valid, raw_x_test = train_data[:,:input_dim], valid_data[:,:input_dim], test_data[:,:input_dim]

    if select_all:
        feat_idx = list(range(raw_x_train.shape[1]))
    else:
        feat_idx = [0,1,2,3,4] # TODO: Select suitable feature columns.
        
    return raw_x_train[:,feat_idx], raw_x_valid[:,feat_idx], raw_x_test[:,feat_idx], y_train, y_valid, y_test

# %% [markdown]
# # Training Loop

# %%
def trainer(train_loader, valid_loader, model, config, device):

    criterion = nn.MSELoss(reduction='mean') # Define your loss function, do not modify this.

    # Define your optimization algorithm. 
    # TODO: Please check https://pytorch.org/docs/stable/optim.html to get more available algorithms.
    # TODO: L2 regularization (optimizer(weight decay...) or implement by your self).
    optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate']) 
    # scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma = 0.2) # Learning rate scheduler.
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=20, verbose=True) # Learning rate scheduler.
    writer = SummaryWriter() # Writer of tensoboard.

    if not os.path.isdir('./models'):
        os.mkdir('./models') # Create directory of saving models.

    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

    trainloss, validloss = [], []
    for epoch in range(n_epochs):
        model.train() # Set your model to train mode.
        loss_record = []

        # train_pbar = tqdm(train_loader, position=0, leave=True) # Progress bar for training.
        train_pbar = train_loader
        for x, y in train_pbar:
            optimizer.zero_grad()               # Set gradient to zero.
            x, y = x.to(device), y.to(device)   # Move your data to device. 
            pred = model(x)             
            loss = criterion(pred, y)
            loss.backward()                     # Compute gradient(backpropagation).
            optimizer.step()                    # Update parameters.
            step += 1
            loss_record.append(loss.detach().item())
            
            # Display current epoch number and loss on tqdm progress bar.
            # train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
            # train_pbar.set_postfix({'loss': loss.detach().item()})

        mean_train_loss = sum(loss_record)/len(loss_record)
        writer.add_scalar('Loss/train', mean_train_loss, step)

        model.eval() # Set your model to evaluation mode.
        loss_record = []
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)

            loss_record.append(loss.item())
            
        mean_valid_loss = sum(loss_record)/len(loss_record)
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
        trainloss.append(mean_train_loss)
        validloss.append(mean_valid_loss)

        writer.add_scalar('Loss/valid', mean_valid_loss, step)
        # scheduler.step() # Update learning rate.
        scheduler.step(mean_valid_loss) # Update learning rate based on validation loss.
            # 打印当前学习率
        current_lr = optimizer.param_groups[0]['lr']
        print(f"Epoch {epoch+1}, 当前学习率: {current_lr:.6f}")

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path']) # Save your best model
            # print('Saving model with loss {:.3f}...'.format(best_loss))
            early_stop_count = 0
        else: 
            early_stop_count += 1

        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            return trainloss, validloss
    return trainloss, validloss




# %%
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = {
    'seed': 3407,      # Your seed number, you can pick your lucky number. :)
    'select_all': True,   # Whether to use all features.
    'valid_ratio': 0.1,   # validation_size = train_size * valid_ratio
    'n_epochs': 500,     # Number of epochs.            
    'batch_size': 256, 
    'learning_rate': 1e-3,              
    'early_stop': 50,    # If model has not improved for this many consecutive epochs, stop training.     
    'save_path': './models/model.ckpt',  # Your model will be saved here.
    'hidden_dim': [200]*5
}


# %%
# Set seed for reproducibility
same_seed(config['seed'])

mat_contents = scipy.io.loadmat('dataset.mat') # Save predictions to a .mat file.
XTrain, XValidation= mat_contents['XTrain'], mat_contents['XValidation']
YTrain, YValidation = mat_contents['YTrain'], mat_contents['YValidation']

print(f""" XTrain size: {XTrain.shape}
      train_data size: {YTrain.shape} 
valid_data size: {YValidation.shape}""")

train_dataset, valid_dataset, test_dataset = myDataset(XTrain, YTrain), \
                                            myDataset(XValidation, YValidation),\
                                            myDataset(XValidation) # Create datasets.


# Pytorch data loader loads pytorch dataset into batches.
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)

hidden_dim = config['hidden_dim'] # You can modify the hidden layers of your model here.
# model = mlp(XTrain.shape[1], YTrain.shape[1], hidden_dim).to(device) # put your model and data on the same computation device.
model = My_Model(XTrain.shape[1], int(YTrain.shape[1]/2), hidden_dim).to(device) # put your model and data on the same computation device.
trainloss, validloss = trainer(train_loader, valid_loader, model, config, device) # Train your model.
preds = predict(test_loader, model, device) 

: 

In [ ]:
scipy.io.savemat('preds.mat', {'preds': preds}) # Save predictions to a .mat file.
# %%
# model_path = './models/model.ckpt' # Load your model from this path.
# model.load_state_dict(torch.load(model_path))
# model.to('cuda') # 如果要在 GPU 上推理
i = 59
plt.plot(preds[i, :], label='Predicted')
plt.plot(YValidation[i, :], label='Ground Truth')
plt.legend()
plt.show()

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=./runs/